In [2]:
pip install networkx[default]


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\dinen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Link Prediction

In [3]:
def split_by_years(reader):
    dict = {}
    for row in reader:
        if row[7] in dict:
            dict[row[7]].append(row)
        else:
            dict[row[7]] = []
    return dict

In [42]:
big_five = ["Premier League", "LaLiga", "Serie A", "Bundesliga", "Ligue 1"]

In [44]:

import networkx as nx
import csv

filename = "top250-00-19.csv"

with open(filename, 'r', encoding="utf8") as csvfile:
    datareader = csv.reader(csvfile)
    next(datareader)

    years = split_by_years(datareader)
    graphs = {}
    for year in years:
        G = nx.DiGraph()
        G.name = year
        for row in years[year]:
            has_from_node = True
            has_to_node = True
            from_team = row[3]
            to_team = row[5]
            if not G.has_node(from_team):
                has_from_node = False
                G.add_node(from_team, league=row[6])
            if not G.has_node(to_team):
                has_to_node = False
                G.add_node(to_team, league=row[6])
            if has_from_node and has_to_node:
                if  G.has_edge(from_team, to_team):
                    weight = G[from_team][to_team]['weight']
                    G[from_team][to_team]['weight'] = weight + int(row[9])
                else :
                    G.add_edge(from_team, to_team, weight=int(row[9]))
            else:
                G.add_edge(from_team, to_team, weight=int(row[9]))
        graphs[year] = G

In [5]:
g = graphs["2000-2001"]
g['Parma']['Lazio']

{'weight': 64310000}

In [24]:
for g in graphs.values():
    in_degree_centrality = nx.in_degree_centrality(g)
    out_degree_centrality = nx.out_degree_centrality(g)
    eigenvector_centrality = nx.eigenvector_centrality(g, max_iter=350)
    print(g.name + " - " + max(in_degree_centrality, key=in_degree_centrality.get) + " : "  + max(out_degree_centrality, key=out_degree_centrality.get) + " : " + max(eigenvector_centrality, key=eigenvector_centrality.get))



2000-2001 - Inter : Atlético Madrid : Leeds
2001-2002 - Inter : Galatasaray : Juventus
2002-2003 - Bay. Leverkusen : Inter : Inter
2003-2004 - Chelsea : Sunderland : Wigan
2004-2005 - FC Porto : Cruzeiro : Real Madrid
2005-2006 - Real Madrid : Spurs : Everton
2006-2007 - Spurs : Chelsea : Valencia CF
2007-2008 - Man City : Liverpool : Middlesbrough
2008-2009 - Spurs : Spurs : Spurs
2009-2010 - Genoa : Real Madrid : Inter
2010-2011 - Genoa : Inter : Man City
2011-2012 - Juventus : Atlético Madrid : Anzhi
2012-2013 - Chelsea : Inter : AC Milan
2013-2014 - SSC Napoli : AS Roma : Liverpool
2014-2015 - Atlético Madrid : Benfica : Arsenal
2015-2016 - Monaco : Spurs : Juventus
2016-2017 - Inter : Newcastle : Crystal Palace
2017-2018 - Everton : Bor. Dortmund : Juventus
2018-2019 - AS Roma : Inter : Bologna


In [94]:
for g in graphs.values():
    betweeness = nx.betweenness_centrality(g)
    print(max(betweeness, key=betweeness.get))

Everton
Parma
Inter
Parma
Liverpool
Spurs
AC Milan
Newcastle
Spurs
Man City
AC Milan
Atlético Madrid
Man City
Juventus
Atlético Madrid
Atlético Madrid
AS Roma
Chelsea
AS Roma


In [7]:
for g in graphs.values():
    communities_generator = nx.community.girvan_newman(g)
    print(len(next(communities_generator)))


14
12
29
25
17
17
18
8
10
11
13
8
8
8
11
5
7
7
7


In [41]:
for g in graphs.values():
    for node in g.nodes:
        print(node)

Parma
Lazio
Arsenal
FC Barcelona
Fiorentina
AS Roma
Real Madrid
Paris SG
West Ham
Leeds
Dep. La Coruña
Real Zaragoza
Monaco
Juventus
Valencia CF
Atlético Madrid
Chelsea
Atlético-PR
Stade Rennais
River Plate
Boca Juniors
Coventry City
Inter
AC Milan
Cagliari Calcio
Bay. Leverkusen
Rangers
Dynamo Kyiv
Spurs
RCD Mallorca
G. Bordeaux
FC Porto
Galatasaray
Benfica
Real Betis
Everton
Sparta Praha
Bor. Dortmund
Málaga CF
Celta de Vigo
Aston Villa
Partizan
PSV Eindhoven
Sporting CP
Middlesbrough
AFC Ajax
Man Utd
São Paulo
Olympique Lyon
Marseille
Racing
Lens
MK Dons
Newcastle
CD Tenerife
Celtic
SSC Napoli
Norwich
Fenerbahce
Liverpool
Internacional
Leicester
Rosenborg BK
Dinamo Zagreb
Feyenoord
Bologna
Grasshoppers
Olympiacos
Bayern Munich 
Villarreal CF
Bolton
Reggina
Fluminense
Lecce
CD Cruz Azul
Hellas Verona
U. de Chile
Ponte Preta
Vitesse
Ipswich
Sunderland
Montpellier
Roda JC
Real Valladolid
Flamengo
Charlton
Newell's
Fulham
Derby
Perugia
Denizlispor
AZ Alkmaar
Real Sociedad
Vélez Sarsfiel

In [65]:
for g in graphs.values():
    out_degree_centrality = nx.out_degree_centrality(g)
    max_out_degree = max(out_degree_centrality, key=out_degree_centrality.get)
    neighbors = g.neighbors(max_out_degree)
    count = 0
    for n in neighbors:
        if(g.nodes[n]["league"] in big_five):
            count = count + 1
        print(g.name + " - Club Name: " + max_out_degree + " Neighbors in the Big Five Leagues: " + count + "out of " + len(neighbors))

    
    
    

2000-2001 - Club Name: Atlético Madrid Neighbors: 
True
True
False
True
True
True
2001-2002 - Club Name: Galatasaray Neighbors: 
True
False
True
True
False
True
True
2002-2003 - Club Name: Inter Neighbors: 
True
True
True
True
False
True
2003-2004 - Club Name: Sunderland Neighbors: 
True
True
True
True
True
True
True
False
2004-2005 - Club Name: Cruzeiro Neighbors: 
False
True
True
True
True
True
2005-2006 - Club Name: Spurs Neighbors: 
True
True
True
True
False
True
2006-2007 - Club Name: Chelsea Neighbors: 
True
True
True
True
True
True
2007-2008 - Club Name: Liverpool Neighbors: 
True
True
False
True
True
True
2008-2009 - Club Name: Spurs Neighbors: 
True
True
True
True
True
2009-2010 - Club Name: Real Madrid Neighbors: 
False
True
True
True
False
2010-2011 - Club Name: Inter Neighbors: 
True
True
False
True
False
2011-2012 - Club Name: Atlético Madrid Neighbors: 
True
True
False
False
False
2012-2013 - Club Name: Inter Neighbors: 
True
True
True
False
True
False
True
True
2013-2014